# Data cleaning and merging dataframes

## Loading multiple datasets

### Classical way

Reading file by file

In [2]:
import pandas as pd

path = 'eniac-data-default/'
orderlines = pd.read_csv(path + 'orderlines.csv')
orders = pd.read_csv(path + 'orders.csv')
brands = pd.read_csv(path + 'brands.csv')
products = pd.read_csv(path + 'products.csv')

In [47]:
path + 'orderlines.csv'

'../data/eniac/orderlines.csv'

### Another way

Using `os` and a loop to read all the files from a directory. It's also possible to read only files with a certain extension (like `.csv`):

In [5]:
import pandas as pd
import os

In [100]:
next(os.walk('./'))

('./',
 ['empty-notebooks', '.ipynb_checkpoints'],
 ['4-initial-exploration-visualisations.ipynb',
  '0-markdown.ipynb',
  '3-data-quality-and-merges.ipynb',
  '1-data-exploration.ipynb',
  'create-product-categories.ipynb',
  '2-data-aggregation.ipynb',
  '3-data-quality-and-merges-guillem.ipynb',
  'Archive.zip',
  '2-data-aggregation-guillem.ipynb',
  '5-category-creation-and-pandas-pipe.ipynb'])

In [6]:
path = '../data/eniac/'
path, dirs, files = next(os.walk(path))
#print(files)

# remove non-csv files
for file in files:
    if not file.endswith("csv"):
        files.remove(file)

In [7]:
files

['products.csv', 'orders.csv', 'orderlines.csv', 'brands.csv']

In [8]:
orderlines.shape

(293983, 7)

In [9]:
# create empty list 
df_list = []

# append datasets to the list
for file in files:
    temp_df = pd.read_csv(path + file, sep=',')
    df_list.append(temp_df)

# show results
products, orders, orderlines, brands = df_list[0], df_list[1], df_list[2], df_list[3]

## Data quality

### Missing values

In [2]:
print("Hi\nI am a new line")

Hi
I am a new line


In [79]:
# products.loc[products.isna().sum(axis=1) > 0].head(30)

In [10]:
def check_missing_values(data): 
    print('Missing values:' + '\n' + str(data.isna().sum()))

In [11]:
for i in range(len(df_list)): 
    print(f'File name: {files[i]}')
    check_missing_values(df_list[i])
    print('\n')

File name: products.csv
Missing values:
sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64


File name: orders.csv
Missing values:
order_id        0
created_date    0
total_paid      5
state           0
dtype: int64


File name: orderlines.csv
Missing values:
id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64


File name: brands.csv
Missing values:
short    0
long     0
dtype: int64




### Duplicates

In [95]:
pd.DataFrame({
    'x':[1,2,3,3,2,3]
})

,x
0,1
1,2
2,3
3,3
4,2
5,3


In [92]:
pd.DataFrame({
    'x':[1,2,3,3,2,3]
}).duplicated().sum()

3

In [84]:
products.duplicated().sum() 

8746

In [86]:
products.shape

(19326, 7)

In [87]:
8746 / 19326

0.4525509676084032

In [12]:
def check_duplicates(data): 
    print('Duplicated rows: ', data.duplicated().sum())

In [13]:
for i in range(len(df_list)): 
    print(f'File name: {files[i]}')
    check_missing_values(df_list[i])
    check_duplicates(df_list[i])
    print('\n')

File name: products.csv
Missing values:
sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64
Duplicated rows:  8746


File name: orders.csv
Missing values:
order_id        0
created_date    0
total_paid      5
state           0
dtype: int64
Duplicated rows:  0


File name: orderlines.csv
Missing values:
id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64
Duplicated rows:  0


File name: brands.csv
Missing values:
short    0
long     0
dtype: int64
Duplicated rows:  0




Based on our initial exploration we need to find a way to deal with our missing values. 

Do we have other problems like data inconsistency? 

* Example: problems with the `price` column in the `orderlines` dataframe.

### Data inconsistency (data types)

In [14]:
for i in range(len(df_list)): 
    
    print(f'File name: {files[i]}')
    check_missing_values(df_list[i])
    check_duplicates(df_list[i])
    df_list[i].info()
    print('\n')

File name: products.csv
Missing values:
sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64
Duplicated rows:  8746
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


File name: orders.csv
Missing values:
order_id        0
created_date    0
total_paid      5
state           0
dtype: int64
Duplicated rows:  0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column     

Some aspects that do not make a lot of sense:

* **products**: 
    * `price` and `promo_price` are loaded as objects. They should be floats. 
    * Quite a lot of missing values: 
        * `description`: 7 missing values. Maybe that could be inferred from the product name?
        * `price`: the missing values could be filled from the `orderlines` dataset. But first we will need to clean it. 

    
* **orders**: 
    * `created_date` should have a date data type. Then, it would be a good quality check to see if the created date for `orders` mathces with the created dates for `orderlines`. 
    
    
* **orderlines**: 
    * `unit_price` has to be a float, something wrong there. 
    * `date` has to be transformed to a date data type. Then, as we said, check that it with matches with the `orders` dataset. 
    
    
* **brands**: looks fine. 


Where do we have to start? Since `orders` and `orderlines` seem very crucial to the analysis, we will start by checking that they match.

## Cleaning orders

In [15]:
# change date datatype
orders['created_date'] = pd.to_datetime(orders['created_date'])

Some data I could extract from here to do a data quality check with `orderlines`: 

* the sum of `total_paid` on both datasets should be the same
* the initial and last date should be the same

In [16]:
min(orders['created_date'])

Timestamp('2017-01-01 00:07:19')

In [17]:
max(orders['created_date'])

Timestamp('2018-03-14 13:58:36')

In [18]:
sum(orders['total_paid'])

nan

I forgot to deal with missing values on our orders dataset!

In [13]:
orders.isna().sum()

order_id        0
created_date    0
total_paid      5
state           0
dtype: int64

In [14]:
# let's filter them out
orders.loc[orders['total_paid'].isna()]

,order_id,created_date,total_paid,state
127701,427314,2017-11-20 18:54:39,NaN,Pending
132013,431655,2017-11-22 12:15:24,NaN,Pending
147316,447411,2017-11-27 10:32:37,NaN,Pending
148833,448966,2017-11-27 18:54:15,NaN,Pending
149434,449596,2017-11-27 21:52:08,NaN,Pending


In [26]:
orders.shape

(226909, 4)

These orders have a state of "Pending", so I can exclude them from our dataset. We are interested on the completed ones.

In [28]:
orders.dropna(inplace=True)

In [29]:
orders.shape

(226904, 4)

Now my dataset is clean. And the total paid is: 

In [23]:
orders['total_paid'].sum()

129159615.07000001

## Cleaning orderlines

First let's transform our date time. 

In [17]:
orderlines['date'] = pd.to_datetime(orderlines['date'])

In [18]:
min(orderlines['date'])
# orders: Timestamp('2017-01-01 00:07:19')

Timestamp('2017-01-01 00:07:19')

In [19]:
max(orderlines['date'])
# orders: Timestamp('2018-03-14 13:58:36')

Timestamp('2018-03-14 13:58:36')

Yes, it seems it match. Now let's test our total_paid. 

To get this value, we will have to calculate a new column, total price for each row. It would be `product_quantity` * `unit_price`. I can not multiply a two different types of data. So I will have to transform the unit price to a numeric one. 

In [51]:
orderlines.sample(5)

,id,id_order,product_id,product_quantity,sku,unit_price,date
6248,1136197,305330,0,1,MOS0174,29.99,2017-01-09 00:56:26
190079,1479793,456449,0,1,BEL0264,14.99,2017-12-06 20:27:28
156455,1427646,434546,0,1,APP2086,1.178.59,2017-11-23 13:18:29
12157,1148536,310553,0,1,APP1611,1.631.99,2017-01-16 16:56:23
50345,1222002,343219,0,1,WAC0228,879.99,2017-04-04 17:20:10


In [20]:
orderlines['product_quantity'] * pd.to_numeric(orderlines['unit_price'])

ValueError: Unable to parse string "1.137.99" at position 6

While trying to transform this column to numeric an error appears. From the error message, "Unable to parse string "1.137.99" at position 6", it seems that our dataset has some problems with the thousands separators: they were encoded as dots, and python only admits one dot per number: the _decimal_ separator!

There are a lot of different ways to solve this problem. The first thing I will do is to count how many dots appear for each number using string operations. If price has more than one dot, I will consider it corrupted. 

In [5]:
# we create a copy of the dataset
ol_temp = orderlines.copy()

In [6]:
orderlines['unit_price'].str.count('\.')

0         1
1         1
2         1
3         1
4         1
         ..
293978    1
293979    1
293980    1
293981    1
293982    1
Name: unit_price, Length: 293983, dtype: int64

In [7]:
# create a new column with the amount of dots in the unit_price column
ol_temp['dots'] = orderlines['unit_price'].str.count('\.')
ol_temp.head(3)

,id,id_order,product_id,product_quantity,sku,unit_price,date,dots
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,1
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,1
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,1


In [8]:
# show the rows with more than one dot
ol_temp.query('dots > 1')

,id,id_order,product_id,product_quantity,sku,unit_price,date,dots
6,1119115,299544,0,1,APP1582,1.137.99,2017-01-01 01:17:21,2
11,1119126,299549,0,1,PAC0929,2.565.99,2017-01-01 02:07:42,2
15,1119131,299553,0,1,APP1854,3.278.99,2017-01-01 02:14:47,2
43,1119195,299582,0,1,PAC0961,2.616.99,2017-01-01 08:54:00,2
59,1119214,299596,0,1,PAC1599,2.873.99,2017-01-01 09:53:11,2
...,...,...,...,...,...,...,...,...
293862,1649999,452946,0,1,APP2075,2.999.00,2018-03-14 13:03:33,2
293887,1650045,527321,0,1,PAC2148,3.497.00,2018-03-14 13:10:15,2
293889,1650050,527324,0,1,PAC2117,3.075.00,2018-03-14 13:10:56,2
293911,1650088,527342,0,1,APP2492,1.329.00,2018-03-14 13:24:51,2


Our theory about the thousands separators is confirmed. How can we solve this problem? 

My approach would be to remove all the dots for all the unit_prices, and then add a dot before the last 2 digits to all the rows. Then I will transform it into numeric values.

In [9]:
orderlines['unit_price'].str.replace('\.','', regex=True)

0          1899
1         39900
2         47405
3          6839
4          2374
          ...  
293978     4299
293979    14158
293980      999
293981     1999
293982     1399
Name: unit_price, Length: 293983, dtype: object

In [10]:
orderlines.sample()

,id,id_order,product_id,product_quantity,sku,unit_price,date
128876,1374409,408389,0,1,BEL0300,35.99,2017-10-09 23:06:36


In [11]:
orderlines = orderlines.assign(unit_price_nd = orderlines['unit_price'].str.replace('\.','', regex=True))
orderlines.head(5)

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,1899
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,39900
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,47405
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,6839
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,2374


In [82]:
orderlines['unit_price_nd'].str[-2:][:5]

0    99
1    00
2    05
3    39
4    74
Name: unit_price_nd, dtype: object

In [83]:
orderlines['unit_price_nd'].str[:-2][:5]

0     18
1    399
2    474
3     68
4     23
Name: unit_price_nd, dtype: object

In [12]:
orderlines['integers'] = orderlines['unit_price_nd'].str[:-2]
orderlines['digits'] = orderlines['unit_price_nd'].str[-2:]
orderlines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd,integers,digits
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,1899,18,99
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,39900,399,00
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,47405,474,05
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,6839,68,39
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,2374,23,74


In [13]:
orderlines['new_unit_price'] = orderlines['integers'] + '.' + orderlines['digits']
orderlines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd,integers,digits,new_unit_price
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,1899,18,99,18.99
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,39900,399,00,399.00
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,47405,474,05,474.05
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,6839,68,39,68.39
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,2374,23,74,23.74


In [87]:
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
 7   unit_price_nd     293983 non-null  object
 8   digits            293983 non-null  object
 9   integers          293983 non-null  object
 10  new_unit_price    293983 non-null  object
dtypes: int64(4), object(7)
memory usage: 24.7+ MB


In [14]:
# last step is to transform it to numeric
orderlines['unit_price'] = pd.to_numeric(orderlines['new_unit_price'])
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                293983 non-null  int64  
 1   id_order          293983 non-null  int64  
 2   product_id        293983 non-null  int64  
 3   product_quantity  293983 non-null  int64  
 4   sku               293983 non-null  object 
 5   unit_price        293983 non-null  float64
 6   date              293983 non-null  object 
 7   unit_price_nd     293983 non-null  object 
 8   integers          293983 non-null  object 
 9   digits            293983 non-null  object 
 10  new_unit_price    293983 non-null  object 
dtypes: float64(1), int64(4), object(6)
memory usage: 24.7+ MB


Now it is time to see if it is the same than the orders files. 

In [ ]:
orderlines.drop(['unit_price_nd','digits','integers','new_unit_price'], axis=1, inplace=True)
orderlines['total_price'] = orderlines['product_quantity'] * orderlines['unit_price']

In [21]:
orderlines.head(10)

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,18.99
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,399.00
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,474.05
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,68.39
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,23.74
5,1119114,295310,0,10,WDT0249,231.79,2017-01-01 01:14:27,2317.90
6,1119115,299544,0,1,APP1582,1137.99,2017-01-01 01:17:21,1137.99
7,1119116,299545,0,1,OWC0100,47.49,2017-01-01 01:46:16,47.49
8,1119119,299546,0,1,IOT0014,18.99,2017-01-01 01:50:34,18.99
9,1119120,295347,0,1,APP0700,72.19,2017-01-01 01:54:11,72.19


In [94]:
orders.head()

,order_id,created_date,total_paid,state
0,241319,2017-01-02 13:35:40,44.99,Cancelled
1,241423,2017-11-06 13:10:02,136.15,Completed
2,242832,2017-12-31 17:40:03,15.76,Completed
3,243330,2017-02-16 10:59:38,84.98,Completed
4,243784,2017-11-24 13:35:19,157.86,Cancelled


In [22]:
sum(orderlines['total_price'])
# 129159615.07000001

128776222.0298095

In [97]:
sum(orders['total_paid'])

129159615.06991275

What!? After all this work the values do not match! How can we figure it out where is the problem? 

## Matching `orders` and `orderlines`

It is possible that some orders exist in one dataset but not in the other one. This would be a potential source for this price mismatch. Let's solve it!

We first create a new column in the `orderlines` dataset using `assign`. We also use `isin()` to create a boolen value (True/False) that checks whether the `id_order` is present in the `orders` dataset:

In [23]:
(
orderlines
    .assign(check_orders = orderlines['id_order'].isin(orders['order_id']))
)['check_orders'].sum()

293749

In [24]:
orderlines.shape

(293983, 8)

Then, using `.query` we select rows where the value in this new column is `False`:

In [25]:
(
orderlines
    .assign(check_orders = orderlines['id_order'].isin(orders['order_id']))
    .query("check_orders==False")
)

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price,check_orders
5,1119114,295310,0,10,WDT0249,231.79,2017-01-01 01:14:27,2317.90,False
63,1119218,296284,0,1,BNQ0042,699.00,2017-01-01 09:58:35,699.00,False
67,1119223,294806,0,1,APP1849,2558.99,2017-01-01 10:09:15,2558.99,False
69,1119226,294806,0,1,APP1864,2797.99,2017-01-01 10:15:14,2797.99,False
70,1119235,297261,0,1,QNA0177,304.99,2017-01-01 10:17:59,304.99,False
...,...,...,...,...,...,...,...,...,...
275665,1621177,244328,0,1,OWC0260,349.00,2018-02-20 13:00:45,349.00,False
280055,1628699,261391,0,1,APP2352,3343.00,2018-02-27 11:43:25,3343.00,False
280352,1629247,287797,0,1,SYN0182,484.11,2018-02-27 19:16:26,484.11,False
280856,1630150,261391,0,1,SAT0091,44.99,2018-02-28 12:18:26,44.99,False


It looks like 234 rows in `orderlines` come from orders not present in the `orders` dataset. This is quite inconsistent, since the `orders` dataset should be the one and only source of truth for orders: if an order is not there, it should not exist. We will try to find out why that happened, but for now, let's just remove those "ghost" orders:

In [105]:
orderlines.shape

(293983, 8)

In [27]:
orderlines = (
orderlines
    .assign(check_orders = orderlines.id_order.isin(orders.order_id))
    .query("check_orders==True")
)

In [28]:
orderlines.shape

(293749, 9)

Now let's look at this problem in the opposite direction: are there orders in the `orders` dataset not prsent in `orderlines`?

In [30]:
(
orders
    .assign(check_orders = orders['order_id'].isin(orderlines['id_order']))
    .query("check_orders==False")
)

,order_id,created_date,total_paid,state,check_orders
8,245941,2017-01-01 10:32:23,183.52,Completed,False
65,268629,2017-01-31 11:27:25,73.23,Completed,False
75,272862,2017-04-18 18:17:10,128.99,Pending,False
82,277655,2017-02-01 08:44:55,9.99,Pending,False
83,277994,2017-01-23 18:30:11,52.99,Completed,False
...,...,...,...,...,...
226835,527328,2018-03-14 13:14:12,0.00,Shopping Basket,False
226851,527344,2018-03-14 13:25:21,0.00,Place Order,False
226853,527346,2018-03-14 13:26:14,0.00,Place Order,False
226855,527348,2018-03-14 13:28:18,0.00,Place Order,False


There are more than 22000 orders in the `orders` dataset that are not present on the `orderlines` dataset!!! We can try to find out why by looking at the state of these orders:

In [31]:
(orders
 .assign(check_orders = orders.order_id.isin(orderlines.id_order))
 .query("check_orders==False")
 ['state'].value_counts()
)

Place Order        12304
Shopping Basket     9810
Completed             45
Cancelled             41
Pending               13
Name: state, dtype: int64

It looks like most of them are orders that were not fully completed: the products were left in the shopping basket or the order was "placed" but maybe not paid (hence the state "Place Order". Some of them were "Completed", though. 

This will require further research, and we might have to come back to these orders if we have to explore consumer behaviour (e.g. why are orders left in the shopping basket?), but for now, for the sake of data quality, let's drop all of these unmatched orders:

In [32]:
orders = (orders.
          assign(check_orders = orders.order_id.isin(orderlines.id_order)).
          query("check_orders==True")
         )

Let's now check again if the total paid matches:

In [33]:
orders.total_paid.sum()

129130877.18

In [34]:
orderlines.total_price.sum()

128662488.13

STILL NOT MATCHING!!! This is outrageous. Let's keep exploring.

## Solving the price mismatch

Let's merge both datasets and compare, order by order, the `total_price`. We will call this new merged dataset `orders_info`.

*Note: To merge `orderlines` with `orders`, we have to group and aggregate `orderlines`!

In [35]:
orders_info = (
orderlines
    .groupby('id_order')
    .agg({'total_price':'sum'})
    .reset_index()
    .merge(orders, how='left', left_on='id_order', right_on='order_id')
    .copy()
)
orders_info

,id_order,total_price,order_id,created_date,total_paid,state,check_orders
0,241319,44.99,241319,2017-01-02 13:35:40,44.99,Cancelled,True
1,241423,129.16,241423,2017-11-06 13:10:02,136.15,Completed,True
2,242832,10.77,242832,2017-12-31 17:40:03,15.76,Completed,True
3,243330,77.99,243330,2017-02-16 10:59:38,84.98,Completed,True
4,243784,153.87,243784,2017-11-24 13:35:19,157.86,Cancelled,True
...,...,...,...,...,...,...,...
204691,527397,42.99,527397,2018-03-14 13:56:38,42.99,Place Order,True
204692,527398,42.99,527398,2018-03-14 13:57:25,42.99,Shopping Basket,True
204693,527399,141.58,527399,2018-03-14 13:57:34,141.58,Shopping Basket,True
204694,527400,19.98,527400,2018-03-14 13:57:41,19.98,Shopping Basket,True


Now that the `total` from both datasets is in the same dataframe, we can create a new column with the difference:

In [36]:
orders_info['price_difference'] = orders_info['total_paid'] - orders_info['total_price']
orders_info.query('state == "Completed"').sample(10)

,id_order,total_price,order_id,created_date,total_paid,state,check_orders,price_difference
122635,435214,843.86,435214,2017-11-24 16:54:15,843.87,Completed,True,0.01
67574,373065,29.99,373065,2017-07-05 16:17:31,36.98,Completed,True,6.99
188413,509096,739.72,509096,2018-02-10 04:52:47,746.71,Completed,True,6.99
60230,365025,678.99,365025,2017-06-19 16:08:33,678.99,Completed,True,0.00
69837,375604,127.99,375604,2017-07-10 21:19:21,134.98,Completed,True,6.99
145494,460556,243.84,460556,2017-12-12 10:42:08,247.83,Completed,True,3.99
70277,376091,99.99,376091,2017-07-11 22:37:09,103.98,Completed,True,3.99
19974,321415,29.99,321415,2017-02-08 17:36:23,34.98,Completed,True,4.99
69281,374980,348.98,374980,2017-07-11 10:11:17,353.97,Completed,True,4.99
70406,376241,344.97,376241,2017-07-12 10:12:49,344.97,Completed,True,0.00


In [127]:
orders_info.price_difference.describe()

count    204691.000000
mean          2.303391
std         297.461827
min      -36159.900000
25%           0.000000
50%           0.000000
75%           0.010000
max       90898.610000
Name: price_difference, dtype: float64

In [156]:
temp = (
orders_info
    .assign(price_diff = round(orders_info['total_paid'] - orders_info['total_price'], 2))
    ['price_diff']
    .unique()
    .tolist()
)

pd.DataFrame({
    'x':temp
}).sort_values(by='x').head(50)

Looks like the maximum and minimun price differences are huge: some orders are really corrupted. But we also see from the mean and the quartiles that the price difference is neglegible for most orders.

## Challenge: Remove outliers

Decide on a criteria for removing orders whenever you cannot trust the price difference between `orders` and `orderlines`. 

In [44]:
orders_info.describe()

,id_order,total_price,order_id,total_paid,price_difference
count,204696.000000,204696.000000,204696.000000,204691.000000,204691.000000
mean,412606.971836,628.553993,412606.971836,630.857620,2.303391
std,65990.293732,1845.109901,65990.293732,1844.405532,297.461827
min,241319.000000,0.000000,241319.000000,0.000000,-36159.900000
25%,355077.750000,49.900000,355077.750000,49.990000,0.000000
50%,412259.500000,149.990000,412259.500000,151.380000,0.000000
75%,469539.250000,664.510000,469539.250000,664.980000,0.010000
max,527401.000000,250802.370000,527401.000000,214747.530000,90898.610000


In [52]:
(
orders_info
    .sort_values('price_difference', ascending=False)
    .tail(30)
)

,id_order,total_price,order_id,created_date,total_paid,state,check_orders,price_difference
11725,312311,164.90,312311,2017-01-20 00:59:54,125.99,Completed,True,-38.91
20654,322147,164.90,322147,2017-02-10 11:56:59,125.99,Completed,True,-38.91
45940,349377,44.99,349377,2017-04-24 21:42:28,4.99,Pending,True,-40.00
8545,308825,164.90,308825,2017-01-13 14:27:46,123.99,Completed,True,-40.91
25517,327363,164.90,327363,2017-02-21 22:42:17,122.99,Completed,True,-41.91
13784,314570,164.90,314570,2017-01-25 09:01:35,122.99,Completed,True,-41.91
21347,322901,164.90,322901,2017-02-12 13:25:12,122.99,Place Order,True,-41.91
986,300474,164.90,300474,2017-01-02 12:24:54,119.00,Completed,True,-45.90
166463,484334,214768.02,484334,2018-01-07 15:58:26,214714.31,Shopping Basket,True,-53.71
54853,359083,13450.87,359083,2017-05-25 13:58:31,13391.86,Shopping Basket,True,-59.01


In [53]:
(
orders_info
    .sort_values('price_difference', ascending=False)
    .head(30)
)

,id_order,total_price,order_id,created_date,total_paid,state,check_orders,price_difference
70971,376851,90.99,376851,2017-07-13 12:58:52,90989.60,Shopping Basket,True,90898.61
32392,334803,79.98,334803,2017-03-13 20:41:31,79981.00,Shopping Basket,True,79901.02
119474,431635,42.99,431635,2017-11-22 12:01:15,42988.90,Shopping Basket,True,42945.91
45740,349166,12.90,349166,2017-04-24 11:40:42,12899.80,Shopping Basket,True,12886.90
12387,313023,10322.92,313023,2017-01-21 21:12:41,14648.91,Shopping Basket,True,4325.99
145,297148,84.55,297148,2017-01-01 16:42:24,4069.54,Completed,True,3984.99
123,293308,66.49,293308,2017-01-01 13:33:43,2635.47,Completed,True,2568.98
191743,512894,1674.60,512894,2018-02-16 13:25:30,3356.18,Completed,True,1681.58
10981,311490,1649.99,311490,2017-01-18 11:02:34,3299.98,Shopping Basket,True,1649.99
188181,508825,1291.60,508825,2018-02-16 11:21:44,2590.18,Completed,True,1298.58


In [50]:
orders_info.isna().sum()

id_order            0
total_price         0
order_id            0
created_date        0
total_paid          0
state               0
check_orders        0
price_difference    0
dtype: int64

In [49]:
orders_info.dropna(inplace=True)

In [61]:
orders_info_cleaned = orders_info.copy()

In [63]:
orders_info_cleaned.drop(orders_info_cleaned[orders_info_cleaned.price_difference < -1000].index, inplace=True)

In [66]:
orders_info_cleaned.drop(orders_info_cleaned[orders_info_cleaned.price_difference > 4000].index, inplace=True)

In [67]:
(
orders_info_cleaned
    .sort_values('price_difference', ascending=False)
    .head()
)

,id_order,total_price,order_id,created_date,total_paid,state,check_orders,price_difference
145,297148,84.55,297148,2017-01-01 16:42:24,4069.54,Completed,True,3984.99
123,293308,66.49,293308,2017-01-01 13:33:43,2635.47,Completed,True,2568.98
191743,512894,1674.60,512894,2018-02-16 13:25:30,3356.18,Completed,True,1681.58
10981,311490,1649.99,311490,2017-01-18 11:02:34,3299.98,Shopping Basket,True,1649.99
188181,508825,1291.60,508825,2018-02-16 11:21:44,2590.18,Completed,True,1298.58


In [68]:
orders_info_cleaned.describe()

,id_order,total_price,order_id,total_paid,price_difference
count,204677.000000,204677.000000,204677.000000,204677.000000,204677.000000
mean,412604.043454,627.208575,412604.043454,628.671667,1.463092
std,65989.522423,1760.153079,65989.522423,1759.873922,12.797880
min,241319.000000,0.000000,241319.000000,0.000000,-381.470000
25%,355076.000000,49.900000,355076.000000,49.990000,0.000000
50%,412252.000000,149.990000,412252.000000,151.390000,0.000000
75%,469536.000000,663.980000,469536.000000,664.950000,0.010000
max,527401.000000,214768.020000,527401.000000,214747.530000,3984.990000


Save the data once you are sure you can trust it!

In [39]:
orderlines.to_csv(path + 'clean/orderlines_cl.csv', index=False)
orders.to_csv(path + 'clean/orders_cl.csv', index=False)

## Challenge: Cleaning products

Now it is time to clean the products dataset. Let's do a quick review of the things where wrong: 

In [43]:
print(products.info())
check_duplicates(products)
check_missing_values(products)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB
None
Duplicated rows:  8746
Missing values:
sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64


Looking at this overview, we can see that there are different things that has to be changed: 

* Data types: 
    * price should be a float
    * promo price should be a float
* Duplicated rows. They have to be removed. 
    * To accomplish this step you can use the method `pd.DataFrame.drop_duplicates()`. Be sure you drop all the duplicates based on the column **sku**, as it is the one you will use to merge with orderlines. 
* Missing values: 
    * Description may be can be infered by the name
    * price. Is there a way we can extract the information from another table?
    * type. Do we need this column for our analysis?
    
This task can be accomoplished by using all the methods you already know.

### Start of the challenge

In [72]:
products_cl = products.drop('type', axis=1)

In [74]:
products_cl = products_cl.drop('in_stock', axis=1)

In [75]:
products_cl = products_cl.drop('promo_price', axis=1)

In [77]:
products_cl = products_cl.assign(price_nd = products_cl['price'].str.replace('\.','', regex=True))

In [79]:
products_cl['integers'] = products_cl['price_nd'].str[:-2]
products_cl['digits'] = products_cl['price_nd'].str[-2:]

In [85]:
list = []
for vali in products_cl['integers']:
    for vald in products_cl['digits']:
        if vali == "":
            list.append(vald)
        else: list.append("vali" + "." + "vald")

In [87]:
products_cl['new_prices'] = list

ValueError: Length of values (373494276) does not match length of index (19326)

## Brands

The brands csv looks fine, so we can work together with it. 

In [45]:
brands.to_csv(path + 'clean/brands.csv', index=False)